In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

sell_in = pd.read_csv('../data/sell-in.txt', sep='\t')
stocks = pd.read_csv('../data/tb_stocks.txt', sep='\t')
productos = pd.read_csv('../data/tb_productos_05262025.txt', sep='\t')
productos_a_predecir = pd.read_csv('../data/product_id_apredecir201912.txt', sep='\t')

# sell_in['periodo_fecha'] = pd.to_datetime(sell_in['periodo'].astype(str), format='%Y%m')
# sell_in['periodo'] = sell_in['periodo_fecha'].dt.strftime('%Y%m')
# stocks['periodo_fecha'] = pd.to_datetime(stocks['periodo'].astype(str), format='%Y%m')
# stocks['periodo'] = stocks['periodo_fecha'].dt.strftime('%Y%m')

In [ ]:
sell_in.info() # 2945818 2293481

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 157.3 MB


In [59]:
productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251 entries, 0 to 1250
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cat1         1251 non-null   object
 1   cat2         1251 non-null   object
 2   cat3         1251 non-null   object
 3   brand        1251 non-null   object
 4   sku_size     1251 non-null   int64 
 5   product_id   1251 non-null   int64 
 6   descripcion  1251 non-null   object
dtypes: int64(2), object(5)
memory usage: 68.5+ KB


In [60]:
productos_a_predecir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   product_id  780 non-null    int64
dtypes: int64(1)
memory usage: 6.2 KB


In [61]:
# obtain all the productos to predict using tb_productos_a_predecir
productos_a_predecir = productos_a_predecir['product_id'].unique()
#filter productos to only those in productos_a_predecir
productos = productos[productos['product_id'].isin(productos_a_predecir)]

In [62]:
productos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 780 entries, 0 to 1250
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cat1         780 non-null    object
 1   cat2         780 non-null    object
 2   cat3         780 non-null    object
 3   brand        780 non-null    object
 4   sku_size     780 non-null    int64 
 5   product_id   780 non-null    int64 
 6   descripcion  780 non-null    object
dtypes: int64(2), object(5)
memory usage: 48.8+ KB


In [63]:
# merge sell_in with productos
sell_in_nuevo = sell_in.merge(productos, on='product_id', how='inner')

In [64]:
sell_in_nuevo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2293481 entries, 0 to 2293480
Data columns (total 13 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
 7   cat1                   object 
 8   cat2                   object 
 9   cat3                   object 
 10  brand                  object 
 11  sku_size               int64  
 12  descripcion            object 
dtypes: float64(2), int64(6), object(5)
memory usage: 227.5+ MB


In [65]:
# select each product_id and order by periodo and customer_id
sell_in_nuevo = sell_in_nuevo.sort_values(by=['product_id', 'periodo', 'customer_id'])

In [66]:
sell_in_nuevo.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion
31064,201701,10001,20001,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
31164,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
31239,201701,10003,20001,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
31163,201701,10004,20001,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
31130,201701,10005,20001,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma


In [72]:
sell_in_nuevo['product_id'].unique().shape

(780,)

In [73]:
sell_in_nuevo['customer_id'].unique().shape

(597,)

In [67]:
# obtain the product 20001 order by client_id and periodo
sell_in_nuevo_20001_10002 = sell_in_nuevo[
    (sell_in_nuevo['product_id'] == 20001) & 
    (sell_in_nuevo['customer_id'] == 10002)
].sort_values(by=['customer_id', 'periodo'])


In [76]:
# for sell_in_nuevo_20001_10002 I need the quantity of rows
sell_in_nuevo_20001_10002.shape[0]


35

In [52]:
sell_in_nuevo_20001_10002

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion
31164,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
86441,201702,10002,20001,0,15,6.79415,6.79415,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
148465,201703,10002,20001,0,22,29.96366,29.94128,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
208737,201704,10002,20001,0,28,23.56126,22.81133,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
271636,201705,10002,20001,0,47,32.85146,31.22847,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
337234,201706,10002,20001,0,61,48.52166,47.57025,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
396858,201707,10002,20001,0,30,22.20691,21.84874,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
457140,201708,10002,20001,0,16,21.61368,17.08052,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
521808,201709,10002,20001,0,49,64.71793,59.17739,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma
588788,201710,10002,20001,0,3,3.31313,3.31313,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma


In [68]:
sell_in_nuevo_20001_10002.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, 31164 to 2260973
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   periodo                35 non-null     int64  
 1   customer_id            35 non-null     int64  
 2   product_id             35 non-null     int64  
 3   plan_precios_cuidados  35 non-null     int64  
 4   cust_request_qty       35 non-null     int64  
 5   cust_request_tn        35 non-null     float64
 6   tn                     35 non-null     float64
 7   cat1                   35 non-null     object 
 8   cat2                   35 non-null     object 
 9   cat3                   35 non-null     object 
 10  brand                  35 non-null     object 
 11  sku_size               35 non-null     int64  
 12  descripcion            35 non-null     object 
dtypes: float64(2), int64(6), object(5)
memory usage: 3.8+ KB


In [69]:
# Asumimos que sell_in_nuevo_20001_10002 ya está filtrado para producto 20001 y cliente 10002
sell_in_nuevo_20001_10002['periodo'] = pd.to_datetime(sell_in_nuevo_20001_10002['periodo'].astype(str), format='%Y%m')

# 1. Rango completo de meses
min_periodo = sell_in_nuevo_20001_10002['periodo'].min()
max_periodo = sell_in_nuevo_20001_10002['periodo'].max()
todos_los_periodos = pd.date_range(start=min_periodo, end=max_periodo, freq='MS')

# 2. DataFrame con todos los meses
df_periodos = pd.DataFrame({'periodo': todos_los_periodos})

# 3. Merge con original
df_completo = df_periodos.merge(
    sell_in_nuevo_20001_10002,
    on='periodo',
    how='left'
)

# 4. Rellenar toneladas en 0 donde falte
df_completo['tn'] = df_completo['tn'].fillna(0)

# 5. Completar las columnas fijas
# df_completo['product_id'] = 20001
# df_completo['customer_id'] = 10002

# 6. Rellenar los campos de referencia con el dato existente
campos_a_rellenar = [
    'plan_precios_cuidados', 'cat1', 'cat2', 'cat3',
    'brand', 'sku_size', 'descripcion'
]
for campo in campos_a_rellenar:
    df_completo[campo] = df_completo[campo].ffill().bfill()

# 7. Rellenar request_qty y request_tn con cero donde falte
df_completo['cust_request_qty'] = df_completo['cust_request_qty'].fillna(0)
df_completo['cust_request_tn'] = df_completo['cust_request_tn'].fillna(0)

# 8. Pasar periodo a string YYYYMM si querés formato compacto
df_completo['periodo'] = df_completo['periodo'].dt.strftime('%Y%m')

In [55]:
df_completo

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion
0,201701,10002,20001,0.0,17.0,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma
1,201702,10002,20001,0.0,15.0,6.79415,6.79415,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma
2,201703,10002,20001,0.0,22.0,29.96366,29.94128,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma
3,201704,10002,20001,0.0,28.0,23.56126,22.81133,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma
4,201705,10002,20001,0.0,47.0,32.85146,31.22847,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma
5,201706,10002,20001,0.0,61.0,48.52166,47.57025,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma
6,201707,10002,20001,0.0,30.0,22.20691,21.84874,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma
7,201708,10002,20001,0.0,16.0,21.61368,17.08052,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma
8,201709,10002,20001,0.0,49.0,64.71793,59.17739,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma
9,201710,10002,20001,0.0,3.0,3.31313,3.31313,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma


In [56]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   periodo                36 non-null     object 
 1   customer_id            36 non-null     int64  
 2   product_id             36 non-null     int64  
 3   plan_precios_cuidados  36 non-null     float64
 4   cust_request_qty       36 non-null     float64
 5   cust_request_tn        36 non-null     float64
 6   tn                     36 non-null     float64
 7   cat1                   36 non-null     object 
 8   cat2                   36 non-null     object 
 9   cat3                   36 non-null     object 
 10  brand                  36 non-null     object 
 11  sku_size               36 non-null     float64
 12  descripcion            36 non-null     object 
dtypes: float64(5), int64(2), object(6)
memory usage: 3.8+ KB
